In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

llm = ChatOpenAI( model_name="gpt-3.5-turbo",temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()])


haiku_template = ChatPromptTemplate.from_messages([
  ("system", "you are the topest Japanese Haiku teller. You create example of Haiku with keyword Human gave you"),
  ("human", "I want to create Haiku about {programming_language}."),
])

haiku_chain = haiku_template | llm

In [2]:
haiku_explain_template = ChatPromptTemplate.from_messages([
  ("system","You are the best analysist of Haiku. You will explain what the Haiku mean which can be understand even by elementary school students"),
  ("human","{explain}"),
])

haiku_explain_chain = haiku_explain_template | llm

final_chain = {"explain" : haiku_chain} | haiku_explain_chain

final_chain.invoke({
  "programming_language" : "Python"
})

In the deep forest,
Python slithers gracefully,
Nature's silent code.This Haiku is about a python moving gracefully in the deep forest. It highlights the beauty and elegance of nature's creatures, like the python, as they move silently through their natural habitat. The poet is appreciating the harmony and balance of the natural world. It teaches us to observe and appreciate the beauty of nature and the creatures that inhabit it.

AIMessageChunk(content="This Haiku is about a python moving gracefully in the deep forest. It highlights the beauty and elegance of nature's creatures, like the python, as they move silently through their natural habitat. The poet is appreciating the harmony and balance of the natural world. It teaches us to observe and appreciate the beauty of nature and the creatures that inhabit it.")